In [ ]:
#Importamos las librerías de pandas
import pandas as pd
import tkinter as tk
from tkinter import ttk, messagebox, font, filedialog
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import pytesseract
import re
import cv2
from PIL import Image
import bcrypt
from sklearn.pipeline import make_pipeline
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input

MAX_LOGIN_ATTEMPTS = 3
login_attempts = 0

# Función para hashear una contraseña
def hash_password(password):
    salt = bcrypt.gensalt()  # Genera un salt
    hashed = bcrypt.hashpw(password.encode('utf-8'), salt)  # Hashea la contraseña
    return hashed

# Función para verificar una contraseña
def verify_password(plain_password, hashed_password):
    return bcrypt.checkpw(plain_password.encode('utf-8'), hashed_password)

# Constantes
MAX_LOGIN_ATTEMPTS = 3
login_attempts = 0

# Función para hashear una contraseña
def hash_password(password):
    salt = bcrypt.gensalt()  # Genera un salt
    hashed = bcrypt.hashpw(password.encode('utf-8'), salt)  # Hashea la contraseña
    return hashed

# Función para verificar una contraseña
def verify_password(plain_password, hashed_password):
    return bcrypt.checkpw(plain_password.encode('utf-8'), hashed_password)

# Cargar y preparar el DataFrame de usuarios
try:
    usuarios = pd.read_csv('usuarios.csv').astype({'Balance': 'float64'})
    usuarios['Balance'] = usuarios['Balance'].astype(float)
    
    # Verificar y crear columnas si no existen
    if 'Gastos_Fijos' not in usuarios.columns:
        usuarios['Gastos_Fijos'] = 0.0
    if 'Gastos_Ocio' not in usuarios.columns:
        usuarios['Gastos_Ocio'] = 0.0
    if 'Ahorro_Inversiones' not in usuarios.columns:
        usuarios['Ahorro_Inversiones'] = 0.0
    
    # Guardar el DataFrame actualizado en el archivo CSV
    usuarios.to_csv('usuarios.csv', index=False)
except Exception as e:
    print(f"Error al cargar usuarios.csv: {str(e)}")
    usuarios = pd.DataFrame(columns=["Nombre del cliente", "I.D.", "Ocupación", 
                                  "Ingreso Mensual", "Meta de Ahorro", "Nombre de usuario", 
                                  "Correo", "Contraseña", "Balance", "Gastos_Fijos", 
                                  "Gastos_Ocio", "Ahorro_Inversiones"]).astype({'Balance': 'float64'})

# Cargar el archivo de gastos
try:
    balance_df = pd.read_csv('gastos.csv').astype({'Monto': 'float64'})
except Exception as e:
    print(f"Error al cargar gastos.csv: {str(e)}")
    balance_df = pd.DataFrame(columns=["Tipo", "I.D.", "Fecha", "Categoría", "Monto", "Descripción"]).astype({'Monto': 'float64'})

# Guardar el archivo actualizado
usuarios.to_csv('usuarios.csv', index=False)

# Cargar el archivo de usuarios
usuarios = pd.read_csv('usuarios.csv')

# Actualizar contraseñas si no están hasheadas
for index, row in usuarios.iterrows():
    if not row['Contraseña'].startswith('$2b$'):  # Verificar si la contraseña no está hasheada
        usuarios.at[index, 'Contraseña'] = hash_password(row['Contraseña']).decode('utf-8')

# Guardar el archivo de usuarios con contraseñas hasheadas
usuarios.to_csv('usuarios.csv', index=False)

def convertir_dolares_a_moneda_local(monto_dolares, tasa_cambio):
    """
    Convierte un monto en dólares a la moneda local usando una tasa de cambio.

    :param monto_dolares: Cantidad en dólares a convertir.
    :param tasa_cambio: Tasa de cambio para convertir a la moneda local.
    :return: Monto convertido a la moneda local.
    """
    return monto_dolares*tasa_cambio

def func_convertir_moneda():
    def procesar_conversion():
        monto_dolares = float(entrada_monto.get())
        tasa_cambio = float(entrada_tasa_cambio.get())
        monto_convertido = convertir_dolares_a_moneda_local(monto_dolares, tasa_cambio)
        messagebox.showinfo("Conversión", f"{monto_dolares} USD equivale a {monto_convertido:.2f} en su moneda local.")

    # Limpiar la ventana antes de mostrar el formulario de conversión
    for widget in windowsmain.winfo_children():
        widget.destroy()

    # Etiqueta y campos para la conversión
    etiqueta_convertir_moneda = tk.Label(windowsmain, text='Convertir Dólares a Moneda Local', font=('Arial', 24))
    etiqueta_convertir_moneda.pack()

    etiqueta_monto = tk.Label(windowsmain, text='Monto en Dólares:', font=('Arial', 16))
    etiqueta_monto.pack()
    entrada_monto = tk.Entry(windowsmain, width=40, font=('Arial', 12))
    entrada_monto.pack()

    etiqueta_tasa_cambio = tk.Label(windowsmain, text='Tasa de Cambio (1 USD = X moneda local):', font=('Arial', 16))
    etiqueta_tasa_cambio.pack()
    entrada_tasa_cambio = tk.Entry(windowsmain, width=40, font=('Arial', 12))
    entrada_tasa_cambio.pack()

    boton_convertir = tk.Button(windowsmain, text='Convertir', command=procesar_conversion)
    boton_convertir.pack()

    boton_volver = tk.Button(windowsmain, text='Volver', command=lambda: func_menuSesionIniciada(usuario_actual))
    boton_volver.pack()

def extraer_texto_de_imagen(ruta_imagen):
    # Cargar la imagen
    imagen = Image.open(ruta_imagen)

    # Convertir la imagen a escala de grises (mejora la precisión de OCR)
    imagen_gris = imagen.convert('L')

    # Usar pytesseract para extraer texto
    texto = pytesseract.image_to_string(imagen_gris)

    return texto

def procesar_texto_recibo(texto):
    # Expresiones regulares para buscar el monto, la fecha y la categoría
    patron_monto = r'\$\s*(\d+\.\d{2})'  # Busca montos como $10.00
    patron_fecha = r'(\d{2}/\d{2}/\d{4})'  # Busca fechas como 12/31/2023
    patron_categoria = r'(comida|transporte|ocio|servicios)'  # Busca categorías comunes

    # Buscar el monto
    monto = re.search(patron_monto, texto)
    monto = float(monto.group(1)) if monto else None

    # Buscar la fecha
    fecha = re.search(patron_fecha, texto)
    fecha = fecha.group(1) if fecha else datetime.now().strftime("%Y-%m-%d")

    # Buscar la categoría
    categoria = re.search(patron_categoria, texto, re.IGNORECASE)
    categoria = categoria.group(1).capitalize() if categoria else "Otros"

    return monto, fecha, categoria

def func_subir_recibo(usuario):
    try:
        # Permitir al usuario seleccionar una imagen
        ruta_imagen = filedialog.askopenfilename(
            title="Seleccionar Recibo",
            filetypes=[("Imágenes", "*.jpg *.jpeg *.png")]
        )

        if not ruta_imagen:
            return  # El usuario canceló la selección

        # Extraer texto de la imagen
        texto = extraer_texto_de_imagen(ruta_imagen)
        print("Texto extraído del recibo:\n", texto)  # Depuración

        # Procesar el texto para obtener monto, fecha y categoría
        monto, fecha, categoria = procesar_texto_recibo(texto)

        if not monto:
            messagebox.showerror("Error", "No se pudo detectar el monto en el recibo.")
            return

        # Obtener el ID del usuario
        id_usuario = usuarios.loc[usuarios['Nombre de usuario'] == usuario, 'I.D.'].values[0]

        # Registrar la transacción
        nueva_transaccion = pd.DataFrame([['Egreso', id_usuario, fecha, categoria, monto, "Recibo procesado"]],
                                         columns=["Tipo", "I.D.", "Fecha", "Categoría", "Monto", "Descripción"])
        global balance_df
        balance_df = pd.concat([balance_df, nueva_transaccion], ignore_index=True)
        balance_df.to_csv("gastos.csv", index=False)

        # Mostrar mensaje de éxito
        messagebox.showinfo("Éxito", f"Transacción registrada:\nMonto: ${monto:.2f}\nFecha: {fecha}\nCategoría: {categoria}")
    except Exception as e:
        messagebox.showerror("Error", f"Ocurrió un error al procesar el recibo: {e}")

def preparar_datos_anomalias(usuario):
    global balance_df

    # Filtrar los egresos del usuario
    id_usuario = usuarios.loc[usuarios['Nombre de usuario'] == usuario, 'I.D.'].values[0]
    egresos = balance_df[(balance_df['I.D.'] == id_usuario) & (balance_df['Tipo'] == 'Egreso')]

    # Crear una serie temporal con los montos
    montos = egresos['Monto'].values.reshape(-1, 1)

    # Normalizar los datos
    scaler = StandardScaler()
    montos_escalados = scaler.fit_transform(montos)

    return montos_escalados, scaler, egresos

def entrenar_autoencoder(datos):
    # Definir la arquitectura del Autoencoder
    input_dim = datos.shape[1]
    encoding_dim = 8  # Dimensión del espacio latente

    # Capa de entrada
    input_layer = Input(shape=(input_dim,))
    # Capa codificadora
    encoder = Dense(encoding_dim, activation="relu")(input_layer)
    # Capa decodificadora
    decoder = Dense(input_dim, activation="sigmoid")(encoder)

    # Modelo Autoencoder
    autoencoder = Model(inputs=input_layer, outputs=decoder)
    autoencoder.compile(optimizer='adam', loss='mean_squared_error')

    # Entrenar el modelo
    autoencoder.fit(datos, datos, epochs=50, batch_size=32, shuffle=True, verbose=0)

    return autoencoder

def detectar_anomalias(autoencoder, datos, umbral=1.5):
    # Predecir los datos reconstruidos
    datos_reconstruidos = autoencoder.predict(datos)

    # Calcular el error de reconstrucción (MSE)
    mse = np.mean(np.power(datos - datos_reconstruidos, 2), axis=1)

    # Identificar anomalías (errores mayores que el umbral)
    anomalias = mse > umbral

    return anomalias, mse

def func_detectar_anomalias(usuario):
    try:
        # Preparar datos
        datos, scaler, egresos = preparar_datos_anomalias(usuario)

        # Verificar si hay suficientes datos
        if len(datos) < 30:  # Mínimo 30 transacciones
            messagebox.showerror("Error", "No hay suficientes datos para detectar anomalías.")
            return

        # Entrenar Autoencoder
        autoencoder = entrenar_autoencoder(datos)

        # Detectar anomalías
        anomalias, mse = detectar_anomalias(autoencoder, datos)

        # Filtrar transacciones anómalas
        egresos['Anomalía'] = anomalias
        transacciones_anomalas = egresos[egresos['Anomalía']]

        # Mostrar resultados en una nueva ventana
        ventana_anomalias = tk.Toplevel(windowsmain)
        ventana_anomalias.title("Anomalías Detectadas")
        ventana_anomalias.geometry("600x400")

        # Crear un Text widget para mostrar las anomalías
        texto_anomalias = tk.Text(ventana_anomalias, wrap=tk.WORD)
        texto_anomalias.pack(expand=True, fill=tk.BOTH, padx=10, pady=10)

        # Mostrar las transacciones anómalas
        if not transacciones_anomalas.empty:
            texto_anomalias.insert(tk.END, "Transacciones Anómalas Detectadas:\n\n")
            for _, transaccion in transacciones_anomalas.iterrows():
                texto_anomalias.insert(tk.END, f"Fecha: {transaccion['Fecha']}\n")
                texto_anomalias.insert(tk.END, f"Monto: ${transaccion['Monto']:.2f}\n")
                texto_anomalias.insert(tk.END, f"Categoría: {transaccion['Categoría']}\n")
                texto_anomalias.insert(tk.END, f"Descripción: {transaccion['Descripción']}\n")
                texto_anomalias.insert(tk.END, "-" * 40 + "\n")
        else:
            texto_anomalias.insert(tk.END, "No se detectaron transacciones anómalas.")

        # Botón para cerrar la ventana
        boton_cerrar = ttk.Button(ventana_anomalias, text="Cerrar", command=ventana_anomalias.destroy)
        boton_cerrar.pack(pady=10)
    except Exception as e:
        messagebox.showerror("Error", f"Ocurrió un error al detectar anomalías: {e}")

def preparar_datos_gastos(usuario):
    global balance_df

    # Filtrar los egresos del usuario
    id_usuario = usuarios.loc[usuarios['Nombre de usuario'] == usuario, 'I.D.'].values[0]
    egresos = balance_df[(balance_df['I.D.'] == id_usuario) & (balance_df['Tipo'] == 'Egreso')]

    # Verificar si hay suficientes datos
    if len(egresos) < 30:  # Mínimo 30 días de datos
        messagebox.showerror("Error", "No hay suficientes datos para realizar la predicción.")
        return

    # Agrupar por fecha y sumar los montos
    egresos['Fecha'] = pd.to_datetime(egresos['Fecha'])
    egresos = egresos.groupby('Fecha')['Monto'].sum().reset_index()

    # Crear una serie temporal con las fechas y montos
    fechas = egresos['Fecha']
    montos = egresos['Monto'].values

    # Normalizar los datos (escalar entre 0 y 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    montos_escalados = scaler.fit_transform(montos.reshape(-1, 1))

    # Crear secuencias de datos para el modelo LSTM
    def crear_secuencias(data, time_step=30):
        X, y = [], []
        for i in range(len(data) - time_step):
            X.append(data[i:(i + time_step), 0])
            y.append(data[i + time_step, 0])
        return np.array(X), np.array(y)

    time_step = 30  # Usar 30 días para predecir el siguiente
    X, y = crear_secuencias(montos_escalados, time_step)

    # Reformatear X para que sea compatible con LSTM [muestras, time_step, características]
    X = X.reshape(X.shape[0], X.shape[1], 1)

    return X, y, scaler, fechas, montos

def entrenar_modelo_lstm(X, y):
    # Crear el modelo LSTM
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))

    # Compilar el modelo
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Entrenar el modelo
    model.fit(X, y, batch_size=32, epochs=20, verbose=1)

    return model

def predecir_gastos_futuros(model, scaler, ultimos_datos, dias_futuros=30):
    # Preparar los últimos datos para la predicción
    ultimos_datos_escalados = scaler.transform(ultimos_datos.reshape(-1, 1))
    X_pred = np.array([ultimos_datos_escalados[-30:]])  # Usar los últimos 30 días
    X_pred = X_pred.reshape(X_pred.shape[0], X_pred.shape[1], 1)

    # Predecir los próximos días
    predicciones = []
    for _ in range(dias_futuros):
        pred = model.predict(X_pred)
        predicciones.append(pred[0, 0])
        X_pred = np.append(X_pred[:, 1:, :], [[pred]], axis=1)

    # Invertir la normalización
    predicciones = scaler.inverse_transform(np.array(predicciones).reshape(-1, 1))

    return predicciones.flatten()

def func_prediccion_gastos(usuario):
    # Preparar datos
    X, y, scaler, fechas, montos = preparar_datos_gastos(usuario)

    # Entrenar modelo (puedes guardar el modelo entrenado para no entrenarlo cada vez)
    model = entrenar_modelo_lstm(X, y)

    # Predecir gastos futuros
    dias_futuros = 30
    predicciones = predecir_gastos_futuros(model, scaler, montos, dias_futuros)

    # Crear fechas futuras
    ultima_fecha = fechas.iloc[-1]
    fechas_futuras = pd.date_range(ultima_fecha, periods=dias_futuros + 1, freq='D')[1:]

    # Graficar resultados
    plt.figure(figsize=(10, 6))
    plt.plot(fechas, montos, label='Gastos Históricos')
    plt.plot(fechas_futuras, predicciones, label='Predicción de Gastos', linestyle='--')
    plt.title('Predicción de Gastos Futuros')
    plt.xlabel('Fecha')
    plt.ylabel('Monto ($)')
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def preparar_datos_clustering():
    global usuarios, balance_df
    
    # Combinar datos de usuarios y gastos
    features = usuarios.copy()
    
    # Calcular % de gasto por categoría para cada usuario
    egresos = balance_df[balance_df["Tipo"] == "Egreso"]
    gastos_por_usuario = egresos.groupby(["I.D.", "Categoría"])["Monto"].sum().unstack(fill_value=0)
    gastos_por_usuario = gastos_por_usuario.div(gastos_por_usuario.sum(axis=1), axis=0)  # Normalizar
    
    # Unir con datos de usuarios (usar ID como índice)
    features = features.set_index("I.D.").merge(gastos_por_usuario, left_index=True, right_index=True, how="left")
    features.fillna(0, inplace=True)
    
    # Seleccionar variables relevantes (ajusta las categorías según tus datos)
    categorias_relevantes = ["Comida", "Transporte", "Entretenimiento", "Servicios"]
    X = features[["Ingreso Mensual", "Meta de Ahorro", "Balance"] + categorias_relevantes]
    
    return X

def entrenar_modelo_clustering():
    X = preparar_datos_clustering()
    
    # Estandarizar datos
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Entrenar modelo K-Means
    kmeans = KMeans(n_clusters=4, random_state=42)
    kmeans.fit(X_scaled)
    
    return kmeans, scaler

def asignar_cluster_usuario(usuario):
    try:
        kmeans, scaler = entrenar_modelo_clustering()
        id_usuario = usuarios.loc[usuarios["Nombre de usuario"] == usuario, "I.D."].values[0]
        usuario_data = preparar_datos_clustering().loc[[id_usuario]]  # Usar ID como índice
        usuario_scaled = scaler.transform(usuario_data)
        return kmeans.predict(usuario_scaled)[0]
    except Exception as e:
        print(f"Error en clustering: {e}")  # Depuración
        return 0  # Cluster por defecto

def obtener_recomendaciones_usuario(usuario):
    cluster = asignar_cluster_usuario(usuario)
    recomendaciones = {
        0: "🔹 Reduce gastos en entretenimiento. Usa el 70% de tu excedente para ahorrar.",
        1: "🔹 Optimiza tus gastos en transporte. Considera opciones más económicas.",
        2: "🔹 Tus hábitos son ejemplares. Invierte el 20% de tus ahorros.",
        3: "🔹 Aumenta tus ingresos pasivos. Revisa guías de inversión."
    }
    return recomendaciones.get(cluster, "🔹 Mantén un registro constante de tus transacciones.")


def graficar_prediccion_balance():
    global balance_df, usuario_actual

    # Obtener datos del usuario actual
    id_usuario = usuarios.loc[usuarios['Nombre de usuario'] == usuario_actual, 'I.D.'].values[0]
    movimientos = balance_df[balance_df['I.D.'] == id_usuario].copy()
    
    if movimientos.empty:
        messagebox.showerror("Error", "No hay datos suficientes para generar una predicción.")
        return plt.figure()  # Retorna figura vacía para evitar errores

    # Convertir fechas y calcular balance acumulado histórico
    movimientos['Fecha'] = pd.to_datetime(movimientos['Fecha'])
    movimientos['Neto'] = movimientos.apply(lambda x: x['Monto'] if x['Tipo'] == 'Ingreso' else -x['Monto'], axis=1)
    balance_acumulado = movimientos.groupby('Fecha')['Neto'].sum().cumsum().reset_index()
    
    # Crear características numéricas (días desde la primera fecha)
    fechas = pd.to_datetime(balance_acumulado['Fecha'])
    balance_acumulado['Dias'] = (fechas - fechas.min()).dt.days
    
    # Entrenar modelo polinomial para el balance
    X = balance_acumulado['Dias'].values.reshape(-1, 1)
    y = balance_acumulado['Neto'].values
    
    model = LinearRegression()
    model.fit(X, y)
    
    # Predecir 30 días futuros
    ultima_fecha = fechas.max()
    dias_futuros = 30
    fechas_futuras = pd.date_range(ultima_fecha, periods=dias_futuros + 1, freq='D')[1:]
    dias_prediccion = (fechas_futuras - fechas.min()).days.values.reshape(-1, 1)
    
    predicciones = model.predict(dias_prediccion)
    
    # Crear gráfico
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(fechas, y, 'o-', label='Histórico')
    ax.plot(fechas_futuras, predicciones, 'r--', label='Predicción')
    ax.set_title('Predicción de Balance a 30 Días', fontsize=14)
    ax.set_xlabel('Fecha', fontsize=12)
    ax.set_ylabel('Balance ($)', fontsize=12)
    ax.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    return fig

def graficar_prediccion_ingresos_egresos(tipo):
    global balance_df, usuario_actual

    # Obtener datos del usuario
    id_usuario = usuarios.loc[usuarios['Nombre de usuario'] == usuario_actual, 'I.D.'].values[0]
    movimientos = balance_df[(balance_df['I.D.'] == id_usuario) & (balance_df['Tipo'] == tipo)].copy()
    
    if movimientos.empty:
        messagebox.showerror("Error", f"No hay suficientes {tipo.lower()}s para predecir.")
        return plt.figure()

    # Procesar fechas y montos
    movimientos['Fecha'] = pd.to_datetime(movimientos['Fecha'])
    movimientos = movimientos.sort_values('Fecha')
    fechas = movimientos['Fecha']
    montos = movimientos['Monto'].values
    
    # Convertir fechas a días desde la primera fecha
    dias = (fechas - fechas.min()).dt.days.values.reshape(-1, 1)
    
    # Entrenar modelo polinomial (grado 2 para capturar tendencias no lineales)
    model = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
    model.fit(dias, montos)
    
    # Predecir 30 días futuros
    ultima_fecha = fechas.max()
    dias_futuros = 30
    fechas_futuras = pd.date_range(ultima_fecha, periods=dias_futuros + 1, freq='D')[1:]
    dias_prediccion = (fechas_futuras - fechas.min()).days.values.reshape(-1, 1)
    
    predicciones = model.predict(dias_prediccion)
    
    # Crear gráfico
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(fechas, montos, 'o-', label='Histórico', color='green' if tipo == 'Ingreso' else 'red')
    ax.plot(fechas_futuras, predicciones, '--', label='Predicción', color='darkgreen' if tipo == 'Ingreso' else 'darkred')
    ax.set_title(f'Predicción de {tipo}s a 30 Días', fontsize=14)
    ax.set_xlabel('Fecha', fontsize=12)
    ax.set_ylabel(f'Monto de {tipo}s ($)', fontsize=12)
    ax.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    return fig

# Función para graficar ingresos vs. egresos
def graficar_relacion_ingresos_egresos():
    global balance_df, usuario_actual

    # Filtrar los datos del usuario actual
    id_usuario = usuarios.loc[usuarios['Nombre de usuario'] == usuario_actual, 'I.D.'].values[0]
    movimientos = balance_df[balance_df['I.D.'] == id_usuario]

    # Convertir la columna 'Fecha' a tipo datetime
    movimientos['Fecha'] = pd.to_datetime(movimientos['Fecha'])

    # Agrupar por fecha y separar ingresos y egresos
    movimientos_agrupados = movimientos.groupby('Fecha')['Monto'].sum().reset_index()

    # Crear el gráfico de dispersión
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.scatter(movimientos_agrupados['Fecha'], movimientos_agrupados['Monto'], color='purple')

    # Etiquetas y título
    ax.set_title('Relación entre Ingresos y Egresos', fontsize=14)
    ax.set_xlabel('Fecha', fontsize=12)
    ax.set_ylabel('Monto ($)', fontsize=12)

    plt.xticks(rotation=45)
    plt.tight_layout()
    return fig

def graficar_distribucion_gastos():
    global balance_df, usuario_actual

    # Filtrar los egresos del usuario actual
    id_usuario = usuarios.loc[usuarios['Nombre de usuario'] == usuario_actual, 'I.D.'].values[0]
    egresos = balance_df[(balance_df['Tipo'] == 'Egreso') & (balance_df['I.D.'] == id_usuario)]

    # Agrupar por categoría y sumar los montos
    gastos_por_categoria = egresos.groupby('Categoría')['Monto'].sum()

    # Crear el gráfico de barras
    fig, ax = plt.subplots(figsize=(10, 6))
    gastos_por_categoria.plot(kind='barh', color='skyblue', ax=ax)

    # Etiquetas y título
    ax.set_title('Distribución de Gastos por Categoría', fontsize=14)
    ax.set_xlabel('Monto ($)', fontsize=12)
    ax.set_ylabel('Categoría', fontsize=12)

    plt.tight_layout()
    return fig

def graficar_proporcion_ingresos_egresos():
    global balance_df, usuario_actual

    # Filtrar los ingresos y egresos del usuario actual
    id_usuario = usuarios.loc[usuarios['Nombre de usuario'] == usuario_actual, 'I.D.'].values[0]
    ingresos = balance_df[(balance_df['Tipo'] == 'Ingreso') & (balance_df['I.D.'] == id_usuario)]
    egresos = balance_df[(balance_df['Tipo'] == 'Egreso') & (balance_df['I.D.'] == id_usuario)]

    # Sumar los montos de ingresos y egresos
    total_ingresos = ingresos['Monto'].sum()
    total_egresos = egresos['Monto'].sum()

    # Crear el gráfico de torta
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.pie([total_ingresos, total_egresos], labels=['Ingresos', 'Egresos'], autopct='%1.1f%%', colors=['lightgreen', 'lightcoral'])
    ax.set_title('Proporción de Ingresos vs. Egresos', fontsize=14)

    return fig

def graficar_evolucion_balance():
    global balance_df, usuario_actual

    # Filtrar los ingresos y egresos del usuario actual
    id_usuario = usuarios.loc[usuarios['Nombre de usuario'] == usuario_actual, 'I.D.'].values[0]
    movimientos = balance_df[balance_df['I.D.'] == id_usuario]

    # Convertir la columna 'Fecha' a tipo datetime
    movimientos['Fecha'] = pd.to_datetime(movimientos['Fecha'])

    # Ordenar por fecha
    movimientos = movimientos.sort_values(by='Fecha')

    # Calcular el balance acumulado
    movimientos['Balance Acumulado'] = movimientos.apply(lambda x: x['Monto'] if x['Tipo'] == 'Ingreso' else -x['Monto'], axis=1).cumsum()

    # Crear el gráfico de líneas
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(movimientos['Fecha'], movimientos['Balance Acumulado'], marker='o', color='blue', linestyle='-', linewidth=2)

    # Etiquetas y título
    ax.set_title('Evolución del Balance en el Tiempo', fontsize=14)
    ax.set_xlabel('Fecha', fontsize=12)
    ax.set_ylabel('Balance ($)', fontsize=12)

    plt.xticks(rotation=45)
    plt.tight_layout()
    return fig

def graficar_comparacion_mensual():
    global balance_df, usuario_actual

    # Filtrar los datos del usuario actual
    id_usuario = usuarios.loc[usuarios['Nombre de usuario'] == usuario_actual, 'I.D.'].values[0]
    movimientos = balance_df[balance_df['I.D.'] == id_usuario]

    # Convertir la columna 'Fecha' a tipo datetime
    movimientos['Fecha'] = pd.to_datetime(movimientos['Fecha'])

    # Extraer el mes y el año
    movimientos['Mes'] = movimientos['Fecha'].dt.to_period('M')

    # Agrupar por mes y tipo, y sumar los montos
    movimientos_agrupados = movimientos.groupby(['Mes', 'Tipo'])['Monto'].sum().unstack()

    # Crear el gráfico de barras apiladas
    fig, ax = plt.subplots(figsize=(10, 6))
    movimientos_agrupados.plot(kind='bar', stacked=True, color=['lightcoral', 'lightgreen'], ax=ax)

    # Etiquetas y título
    ax.set_title('Comparación Mensual de Ingresos y Egresos', fontsize=14)
    ax.set_xlabel('Mes', fontsize=12)
    ax.set_ylabel('Monto ($)', fontsize=12)

    plt.xticks(rotation=45)
    plt.tight_layout()
    return fig


def func_iniciarSesion():
    global usuario_actual

    def func_autenticacion():
        global usuario_actual, login_attempts

        usuario = entradas[0].get().strip() # Obtener el texto de la primera entrada (Nombre de usuario)
        contraseña = entradas[1].get().strip() # Obtener el texto de la segunda entrada (Contraseña)

        if login_attempts >= MAX_LOGIN_ATTEMPTS:
            messagebox.showerror('Error', 'Demasiados intentos fallidos. Intente más tarde.')
            return

        # Buscar el usuario en el DataFrame
        if usuario in usuarios['Nombre de usuario'].values:
            index = usuarios.loc[usuarios['Nombre de usuario'] == usuario].index[0]
            hashed_password = usuarios.loc[index, 'Contraseña'].encode('utf-8')

            if verify_password(contraseña, hashed_password):
                login_attempts = 0  # Reiniciar contador
                func_menuSesionIniciada(usuario)
            else:
                login_attempts += 1
                messagebox.showerror('Error', f"Contraseña incorrecta. Intentos restantes: {MAX_LOGIN_ATTEMPTS - login_attempts}")
        else:
            messagebox.showerror('Error', "Usuario no encontrado")

    for widget in windowsmain.winfo_children():
        widget.destroy()

    etiqueta_iniciarSesion_bienvenida = tk.Label(windowsmain, text='Iniciar sesión', font=('Arial', 24))
    etiqueta_iniciarSesion_bienvenida.pack()

    etiquetas = [
        'Nombre de usuario',
        'Contraseña'
    ]

    entradas = []

    for etiqueta in etiquetas:
        label = tk.Label(windowsmain, text=etiqueta, font=('Arial', 16))
        label.pack()
        entry = tk.Entry(windowsmain, width=40, font=('Arial', 12))
        entry.pack()

        entradas.append(entry)
    
    boton_iniciarSesion_procesar = tk.Button(windowsmain, text='Iniciar sesión', command=func_autenticacion)
    boton_iniciarSesion_procesar.pack()

    boton_iniciarSesion_volver = tk.Button(windowsmain, text='Volver', command=func_menuPrincipal)
    boton_iniciarSesion_volver.pack()


def func_ingresar_ingreso():
    global usuario_actual

    def guardar_ingreso():
        global balance_df, usuarios, etiqueta_balance_actual

        estilo = ttk.Style()
        estilo.configure("Titulo.TLabel", font=('Arial', 18), background='#f0f0f0', foreground='#333333')
        estilo.configure("Subtitulo.TLabel", font=('Arial', 14), background='#f0f0f0', foreground='#333333')
        estilo.configure("Texto.TLabel", font=('Arial', 12), background='#f0f0f0', foreground='#666666')
        estilo.configure("TButton", font=('Arial', 12), padding=5)

        
        # Validar categoría
        categoria = combo_categoria.get().strip()
        if not categoria or len(categoria) > 50:
            messagebox.showerror('Error', 'La categoría no puede estar vacía o tener más de 50 caracteres.')
            return

        # Validar monto
        monto = entrada_monto.get().strip()
        try:
            monto = float(monto)
            if monto <= 0:
                raise ValueError
        except ValueError:
            messagebox.showerror('Error', 'El monto debe ser un número positivo.')
            return
        
        # Obtener los porcentajes
        porcentaje_fijos = scale_fijos.get()
        porcentaje_ocio = scale_ocio.get()
        porcentaje_ahorro = scale_ahorro.get()

        # Validar que la suma sea 100%
        if porcentaje_fijos + porcentaje_ocio + porcentaje_ahorro != 100:
            messagebox.showerror('Error', 'La suma de los porcentajes debe ser 100%.')
            return
        
        # Calcular los montos para cada categoría
        monto_fijos = monto * (porcentaje_fijos / 100)
        monto_ocio = monto * (porcentaje_ocio / 100)
        monto_ahorro = monto * (porcentaje_ahorro / 100)

        # Obtener el ID del usuario actual
        id_usuario = usuarios.loc[usuarios['Nombre de usuario'] == usuario_actual, 'I.D.'].values[0]

        # Actualizar el balance total
        usuarios.loc[usuarios['Nombre de usuario'] == usuario_actual, 'Balance'] += monto

        # Verificar y reemplazar NaN por 0.0
        usuarios['Gastos_Fijos'] = usuarios['Gastos_Fijos'].fillna(0.0)
        usuarios['Gastos_Ocio'] = usuarios['Gastos_Ocio'].fillna(0.0)
        usuarios['Ahorro_Inversiones'] = usuarios['Ahorro_Inversiones'].fillna(0.0)

        # Obtener el índice del usuario actual
        indice_usuario = usuarios.loc[usuarios['Nombre de usuario'] == usuario_actual].index[0]

        # Actualizar las sub-billeteras
        usuarios.at[indice_usuario, 'Gastos_Fijos'] += monto_fijos
        usuarios.at[indice_usuario, 'Gastos_Ocio'] += monto_ocio
        usuarios.at[indice_usuario, 'Ahorro_Inversiones'] += monto_ahorro

        # Guardar los cambios en el archivo CSV
        usuarios.to_csv('usuarios.csv', index=False)

        # Validar descripción
        descripcion = entrada_descripcion.get().strip()
        if not descripcion or len(descripcion) > 100:
            messagebox.showerror('Error', 'La descripción no puede estar vacía o tener más de 100 caracteres.')
            return
        
        confirmacion = messagebox.askyesno('Confirmar', '¿Estás seguro de que deseas registrar este ingreso?')
        if not confirmacion:
            return

        # Si todas las validaciones pasan, guardar el ingreso
        fecha = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        id_usuario = usuarios.loc[usuarios['Nombre de usuario'] == usuario_actual, 'I.D.'].values[0]

        nuevo_ingreso = pd.DataFrame([['Ingreso', id_usuario, fecha, categoria, monto, descripcion]],
                                    columns=["Tipo", "I.D.", "Fecha", "Categoría", "Monto", "Descripción"])
        balance_df = pd.concat([balance_df, nuevo_ingreso], ignore_index=True)
        balance_df.to_csv("gastos.csv", index=False)

        # Actualizar la etiqueta de balance en la interfaz
        try:
            balance_actual = usuarios.loc[usuarios['Nombre de usuario'] == usuario_actual, 'Balance'].values[0]
            etiqueta_balance_actual.config(text=f'Balance actual: ${balance_actual:.2f}')
        except tk.TclError as e:
            print(f"Error al actualizar la etiqueta: {e}")

        # Actualizar el historial de ingresos
        actualizar_historial()

        # Mostrar mensaje de éxito
        etiqueta_exito = ttk.Label(windowsmain, text='¡Ingreso registrado con éxito!', style="Exito.TLabel")
        etiqueta_exito.grid(row=6, column=0, columnspan=2, pady=10)
        messagebox.showinfo('Ingreso guardado', '¡Ingreso registrado con éxito!')

    def actualizar_historial():
        # Obtener los últimos 5 ingresos del usuario
        id_usuario = usuarios.loc[usuarios['Nombre de usuario'] == usuario_actual, 'I.D.'].values[0]
        ingresos_recientes = balance_df[(balance_df['I.D.'] == id_usuario) & (balance_df['Tipo'] == 'Ingreso')].tail(5)

        # Limpiar el historial
        for widget in frame_historial.winfo_children():
            widget.destroy()

        # Mostrar el historial
        ttk.Label(frame_historial, text='Historial de Ingresos Recientes:', style="Titulo.TLabel").pack(pady=10)
        for _, ingreso in ingresos_recientes.iterrows():
            ttk.Label(frame_historial, text=f"{ingreso['Fecha']} - {ingreso['Categoría']}: ${ingreso['Monto']:.2f}", style="Texto.TLabel").pack()

    def actualizar_grafico():
        fig, ax = plt.subplots(figsize=(4, 4))
        categorias = ['Gastos Fijos', 'Gastos de Ocio', 'Ahorro e Inversiones']
        porcentajes = [scale_fijos.get(), scale_ocio.get(), scale_ahorro.get()]
        ax.pie(porcentajes, labels=categorias, autopct='%1.1f%%', colors=['lightblue', 'lightgreen', 'lightcoral'])
        ax.set_title('Distribución del Ingreso')

        # Mostrar el gráfico en la interfaz
        for widget in frame_grafico.winfo_children():
            widget.destroy()
        canvas = FigureCanvasTkAgg(fig, frame_grafico)
        canvas.get_tk_widget().pack()

    # Limpiar la ventana antes de mostrar el formulario de ingreso
    for widget in windowsmain.winfo_children():
        widget.destroy()

    # Crear un Canvas y un Scrollbar
    canvas = tk.Canvas(windowsmain)
    scrollbar = ttk.Scrollbar(windowsmain, orient="vertical", command=canvas.yview)
    scrollable_frame = ttk.Frame(canvas)

    # Configurar el Canvas y el Scrollbar
    scrollable_frame.bind(
        "<Configure>",
        lambda e: canvas.configure(
            scrollregion=canvas.bbox("all")
        )
    )

    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    canvas.configure(yscrollcommand=scrollbar.set)

    canvas.pack(side="left", fill="both", expand=True)
    scrollbar.pack(side="right", fill="y")

    # Frame para el historial de ingresos
    frame_historial = ttk.Frame(scrollable_frame)
    frame_historial.pack(pady=10)
    actualizar_historial()

    # Etiqueta y campos para ingresar el ingreso
    etiqueta_ingresar_ingreso = ttk.Label(scrollable_frame, text='Registrar Ingreso', style="Titulo.TLabel")
    etiqueta_ingresar_ingreso.pack(pady=20)

    # Menú desplegable para categorías
    frame_categoria = ttk.Frame(scrollable_frame)
    frame_categoria.pack(pady=5, anchor="w")

    etiqueta_categoria = ttk.Label(frame_categoria, text='Categoría:', style="Texto.TLabel")
    etiqueta_categoria.pack(side="left", padx=(215, 25))  # Alineado a la izquierda con un pequeño margen

    categorias = ['Salario', 'Bonificación', 'Inversiones', 'Beca Universitaria', 'Otros']
    combo_categoria = ttk.Combobox(frame_categoria, values=categorias, font=('Arial', 12))
    combo_categoria.pack(side="left")  # Alineado a la izquierda, junto a la etiqueta

    # Monto
       
    frame_monto = ttk.Frame(scrollable_frame)
    frame_monto.pack(pady=5, anchor="w")

    etiqueta_monto = ttk.Label(frame_monto, text='Monto:', style="Texto.TLabel")
    etiqueta_monto.pack(side="left", padx=(215, 50))  # Alineado a la izquierda con un pequeño margen

    entrada_monto = ttk.Entry(frame_monto, width=20, font=('Arial', 12))
    entrada_monto.pack(side="left")  # Alineado a la izquierda, junto a la etiqueta

    frame_descripcion = ttk.Frame(scrollable_frame)
    frame_descripcion.pack(pady=5, anchor="w")

    etiqueta_descripcion = ttk.Label(frame_descripcion, text='Descripción:', style="Texto.TLabel")
    etiqueta_descripcion.pack(side="left", padx=(215, 10))

    entrada_descripcion = ttk.Entry(frame_descripcion, width=40, font=('Arial', 12))
    entrada_descripcion.pack(side="left")

    # Barras deslizantes para distribuir el ingreso
    etiqueta_distribucion = ttk.Label(scrollable_frame, text='Distribución del Ingreso:', style="Subtitulo.TLabel")
    etiqueta_distribucion.pack(pady=30)

    etiqueta_fijos = ttk.Label(scrollable_frame, text='Gastos Fijos:', style="Texto.TLabel")
    etiqueta_fijos.pack()
    scale_fijos = ttk.Scale(scrollable_frame, from_=0, to=100, orient='horizontal', length=300)
    scale_fijos.set(50)  # Valor por defecto
    scale_fijos.pack()

    etiqueta_ocio = ttk.Label(scrollable_frame, text='Gastos de Ocio:', style="Texto.TLabel")
    etiqueta_ocio.pack()
    scale_ocio = ttk.Scale(scrollable_frame, from_=0, to=100, orient='horizontal', length=300)
    scale_ocio.set(30)  # Valor por defecto
    scale_ocio.pack()

    etiqueta_ahorro = ttk.Label(scrollable_frame, text='Ahorro e Inversiones:', style="Texto.TLabel")
    etiqueta_ahorro.pack()
    scale_ahorro = ttk.Scale(scrollable_frame, from_=0, to=100, orient='horizontal', length=300)
    scale_ahorro.set(20)  # Valor por defecto
    scale_ahorro.pack()

    # Frame para el gráfico
    frame_grafico = ttk.Frame(scrollable_frame)
    frame_grafico.pack(pady=10)
    actualizar_grafico()

    # Actualizar el gráfico cuando se muevan las barras deslizantes
    scale_fijos.config(command=lambda _: actualizar_grafico())
    scale_ocio.config(command=lambda _: actualizar_grafico())
    scale_ahorro.config(command=lambda _: actualizar_grafico())

    etiqueta_recomendacion = ttk.Label(scrollable_frame, text='Recomendación: 50%-60% Gastos Fijos, \n20%-30% Gastos de Ocio, 10%-20% Ahorro e Inversiones', style="Texto.TLabel")
    etiqueta_recomendacion.pack()

 

    boton_guardar_ingreso = ttk.Button(scrollable_frame, text='Guardar Ingreso', command=guardar_ingreso, style="TButton")
    boton_guardar_ingreso.pack(pady=10)

    boton_volver = ttk.Button(scrollable_frame, text='Volver', command=lambda: func_menuSesionIniciada(usuario_actual), style="TButton")
    boton_volver.pack(pady=20)


def func_ingresar_egreso():
    global usuario_actual, etiqueta_balance_actual

    estilo = ttk.Style()
    estilo.configure("Titulo.TLabel", font=('Arial', 18), background='#f0f0f0', foreground='#333333')
    estilo.configure("Subtitulo.TLabel", font=('Arial', 14), background='#f0f0f0', foreground='#333333')
    estilo.configure("Texto.TLabel", font=('Arial', 12), background='#f0f0f0', foreground='#666666')
    estilo.configure("TButton", font=('Arial', 12), padding=5)

    def guardar_egreso():
        print("Función guardar_egreso ejecutada")  # Mensaje de depuración
        global balance_df, usuarios

        # Validar categoría
        categoria = combo_categoria.get().strip()
        print(f"Categoría seleccionada: {categoria}")  # Mensaje de depuración
        if categoria not in ['Gastos Fijos', 'Gastos Ocio', 'Ahorro e Inversiones']:
            messagebox.showerror('Error', 'La categoría no es válida.')
            return

        # Validar monto
        monto = entrada_monto.get().strip()
        print(f"Monto ingresado: {monto}")  # Mensaje de depuración
        try:
            monto = float(monto)
            if monto <= 0:
                raise ValueError
        except ValueError:
            messagebox.showerror('Error', 'El monto debe ser un número positivo.')
            return

        # Obtener el ID del usuario actual
        id_usuario = usuarios.loc[usuarios['Nombre de usuario'] == usuario_actual, 'I.D.'].values[0]

        # Verificar que haya suficiente saldo en la sub-billetera correspondiente
        columna_sub_billetera = categoria.replace(' ', '_')
        saldo_actual = usuarios.loc[usuarios['Nombre de usuario'] == usuario_actual, columna_sub_billetera].values[0]

        if monto > saldo_actual:
            messagebox.showerror('Error', f'No hay suficiente saldo en {categoria}.')
            return

        # Restar el monto de la sub-billetera correspondiente
        usuarios.loc[usuarios['Nombre de usuario'] == usuario_actual, columna_sub_billetera] -= monto

        # Restar el monto del balance total
        usuarios.loc[usuarios['Nombre de usuario'] == usuario_actual, 'Balance'] -= monto

        # Guardar los cambios en el archivo CSV de usuarios
        usuarios.to_csv('usuarios.csv', index=False)

        # Validar descripción
        descripcion = entrada_descripcion.get().strip()
        if not descripcion or len(descripcion) > 100:
            messagebox.showerror('Error', 'La descripción no puede estar vacía o tener más de 100 caracteres.')
            return
        
        confirmacion = messagebox.askyesno('Confirmar', '¿Estás seguro de que deseas registrar este egreso?')
        if not confirmacion:
            return

        # Registrar el egreso en el archivo de transacciones (gastos.csv)
        fecha = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        descripcion = entrada_descripcion.get().strip()
        nuevo_egreso = pd.DataFrame([['Egreso', id_usuario, fecha, categoria, monto, descripcion]],
                                    columns=["Tipo", "I.D.", "Fecha", "Categoría", "Monto", "Descripción"])

        # Verificar si el archivo gastos.csv existe y cargarlo
        try:
            balance_df = pd.read_csv("gastos.csv")
        except FileNotFoundError:
            # Si el archivo no existe, crear un DataFrame vacío con las columnas necesarias
            balance_df = pd.DataFrame(columns=["Tipo", "I.D.", "Fecha", "Categoría", "Monto", "Descripción"])

        # Concatenar el nuevo egreso
        balance_df = pd.concat([balance_df, nuevo_egreso], ignore_index=True)

        # Guardar el DataFrame actualizado en el archivo CSV
        balance_df.to_csv("gastos.csv", index=False)

        # Actualizar la etiqueta de balance en la interfaz
        balance_actual = usuarios.loc[usuarios['Nombre de usuario'] == usuario_actual, 'Balance'].values[0]
        etiqueta_balance_actual.config(text=f'Balance actual: ${balance_actual:.2f}')

        # Actualizar el historial de egresos y los balances de las sub-billeteras
        actualizar_historial()
        mostrar_sub_billeteras()

        # Mostrar mensaje de éxito
        messagebox.showinfo('Egreso guardado', '¡Egreso registrado con éxito!')

    def actualizar_historial():
        # Obtener los últimos 5 egresos del usuario
        id_usuario = usuarios.loc[usuarios['Nombre de usuario'] == usuario_actual, 'I.D.'].values[0]
        egresos_recientes = balance_df[(balance_df['I.D.'] == id_usuario) & (balance_df['Tipo'] == 'Egreso')].tail(5)

        # Limpiar el historial
        for widget in frame_historial.winfo_children():
            widget.destroy()

        # Mostrar el historial
        tk.Label(frame_historial, text='Historial de Egresos Recientes:', font=('Arial', 14)).pack()
        for _, egreso in egresos_recientes.iterrows():
            tk.Label(frame_historial, text=f"{egreso['Fecha']} - {egreso['Categoría']}: ${egreso['Monto']:.2f}", font=('Arial', 12)).pack()

    def mostrar_sub_billeteras():
        # Obtener los montos de las sub-billeteras
        gastos_fijos = usuarios.loc[usuarios['Nombre de usuario'] == usuario_actual, 'Gastos_Fijos'].values[0]
        gastos_ocio = usuarios.loc[usuarios['Nombre de usuario'] == usuario_actual, 'Gastos_Ocio'].values[0]
        ahorro_inversiones = usuarios.loc[usuarios['Nombre de usuario'] == usuario_actual, 'Ahorro_Inversiones'].values[0]

        # Mostrar los montos en la interfaz
        etiqueta_gastos_fijos.config(text=f'Gastos Fijos: ${gastos_fijos:.2f}')
        etiqueta_gastos_ocio.config(text=f'Gastos Ocio: ${gastos_ocio:.2f}')
        etiqueta_ahorro_inversiones.config(text=f'Ahorro e Inversiones: ${ahorro_inversiones:.2f}')

    # Limpiar la ventana antes de mostrar el formulario de egreso
    for widget in windowsmain.winfo_children():
        widget.destroy()

    # Crear un Canvas y un Scrollbar
    canvas = tk.Canvas(windowsmain)
    scrollbar = tk.Scrollbar(windowsmain, orient="vertical", command=canvas.yview)
    scrollable_frame = tk.Frame(canvas)

    # Configurar el Canvas y el Scrollbar
    scrollable_frame.bind(
        "<Configure>",
        lambda e: canvas.configure(
            scrollregion=canvas.bbox("all")
        )
    )

    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    canvas.configure(yscrollcommand=scrollbar.set)

    canvas.pack(side="left", fill="both", expand=True)
    scrollbar.pack(side="right", fill="y")

    # Frame para el historial de egresos
    frame_historial = tk.Frame(scrollable_frame)
    frame_historial.pack(pady=10)
    actualizar_historial()

    # Etiqueta y campos para ingresar el egreso
    etiqueta_ingresar_egreso = ttk.Label(scrollable_frame, text='Registrar Egreso', font=('Arial', 14))
    etiqueta_ingresar_egreso.pack()

    # Botón para subir recibos
    boton_subir_recibo = ttk.Button(
        scrollable_frame,
        text='Subir Recibo',
        command=lambda: func_subir_recibo(usuario_actual),
        style="TButton",
    )
    boton_subir_recibo.pack(pady=10)

    # Menú desplegable para categorías
    frame_categoria = ttk.Frame(scrollable_frame)
    frame_categoria.pack(pady=5, anchor="w")

    etiqueta_categoria = ttk.Label(frame_categoria, text='Categoría:', style="Texto.TLabel")
    etiqueta_categoria.pack(side="left", padx=(100, 25), pady=(20))  # Alineado a la izquierda con un pequeño margen

    categorias = ['Gastos Fijos', 'Gastos Ocio', 'Ahorro e Inversiones']
    combo_categoria = ttk.Combobox(frame_categoria, values=categorias, font=('Arial', 12))
    combo_categoria.pack(side="left")  # Alineado a la izquierda, junto a la etiqueta

    # Monto 
    frame_monto = ttk.Frame(scrollable_frame)
    frame_monto.pack(pady=5, anchor="w")

    etiqueta_monto = ttk.Label(frame_monto, text='Monto:', style="Texto.TLabel")
    etiqueta_monto.pack(side="left", padx=(100, 50))  # Alineado a la izquierda con un pequeño margen

    entrada_monto = ttk.Entry(frame_monto, width=20, font=('Arial', 12))
    entrada_monto.pack(side="left")  # Alineado a la izquierda, junto a la etiqueta

    frame_descripcion = ttk.Frame(scrollable_frame)
    frame_descripcion.pack(pady=5, anchor="w")

    etiqueta_descripcion = ttk.Label(frame_descripcion, text='Descripción:', style="Texto.TLabel")
    etiqueta_descripcion.pack(side="left", padx=(100, 10))

    entrada_descripcion = ttk.Entry(frame_descripcion, width=30, font=('Arial', 12))
    entrada_descripcion.pack(side="left")

    # Mostrar los balances de las sub-billeteras
    etiqueta_gastos_fijos = ttk.Label(scrollable_frame, text='Gastos Fijos: $0.00', style="Texto.TLabel")
    etiqueta_gastos_fijos.pack(pady=20)

    etiqueta_gastos_ocio = ttk.Label(scrollable_frame, text='Gastos Ocio: $0.00', style="Texto.TLabel")
    etiqueta_gastos_ocio.pack()

    etiqueta_ahorro_inversiones = ttk.Label(scrollable_frame, text='Ahorro e Inversiones: $0.00', style="Texto.TLabel")
    etiqueta_ahorro_inversiones.pack()

    # Mostrar los valores actuales de las sub-billeteras
    mostrar_sub_billeteras()

    boton_guardar_egreso = ttk.Button(scrollable_frame, text='Guardar Egreso', command=guardar_egreso, style="TButton")
    boton_guardar_egreso.pack(pady=10)

    boton_volver = ttk.Button(scrollable_frame, text='Volver', command=lambda: func_menuSesionIniciada(usuario_actual), style="TButton")
    boton_volver.pack(pady=10)

def visualizar_grafica_seleccionada(opcion):
    global balance_df, usuario_actual

    # Verificar si hay datos para graficar
    id_usuario = usuarios.loc[usuarios['Nombre de usuario'] == usuario_actual, 'I.D.'].values[0]
    movimientos = balance_df[balance_df['I.D.'] == id_usuario]

    if movimientos.empty:
        messagebox.showerror('Error', 'No hay datos para graficar.')
        return
    
    if opcion == "Distribución de Gastos":
        fig = graficar_distribucion_gastos()
    elif opcion == "Proporción Ingresos vs. Egresos":
        fig = graficar_proporcion_ingresos_egresos()
    elif opcion == "Evolución del Balance":
        fig = graficar_evolucion_balance()
    elif opcion == "Comparación Mensual":
        fig = graficar_comparacion_mensual()
    elif opcion == "Relación Ingresos vs. Egresos":
        fig = graficar_relacion_ingresos_egresos()
    elif opcion == "Predicción de Balance":
        fig = graficar_prediccion_balance()
    elif opcion == "Predicción de Ingresos":
        fig = graficar_prediccion_ingresos_egresos('Ingreso')
    elif opcion == "Predicción de Egresos":
        fig = graficar_prediccion_ingresos_egresos('Egreso')
    else:
        messagebox.showerror("Error", "Gráfica no válida")
        return

    # Crear una nueva ventana para el gráfico
    ventana_grafico = tk.Toplevel(windowsmain)
    ventana_grafico.title(opcion)

    # Convertir el gráfico de Matplotlib a un objeto Tkinter
    canvas = FigureCanvasTkAgg(fig, ventana_grafico)
    canvas.get_tk_widget().pack()

    # Mostrar el gráfico
    canvas.draw()

def func_menuSesionIniciada(usuario):
    global usuario_actual, etiqueta_balance_actual

    # Verificar si el usuario existe en la base de datos
    if usuario not in usuarios['Nombre de usuario'].values:
        messagebox.showerror("Error", "Usuario no encontrado")
        return  # Salir de la función si el usuario no existe

    print(f"Ejecutando func_menuSesionIniciada para el usuario: {usuario}")  # Debug
    usuario_actual = usuario

    # Limpiar la ventana antes de mostrar el nuevo menú
    for widget in windowsmain.winfo_children():
        widget.destroy()

    # Obtener el balance actual del usuario
    balance_actual = usuarios.loc[usuarios['Nombre de usuario'] == usuario_actual, 'Balance'].values[0]

    # Crear un frame principal para organizar los elementos
    frame_principal = tk.Frame(windowsmain, bg='#f0f0f0')
    frame_principal.pack(expand=True, fill='both', padx=20, pady=20)

    # Etiqueta de bienvenida
    etiqueta_menuSesionIniciada_bienvenida = tk.Label(
        frame_principal,
        text=f'Bienvenido {usuario}',
        font=('Arial', 24, 'bold'),
        bg='#f0f0f0'
    )
    etiqueta_menuSesionIniciada_bienvenida.pack(pady=(10, 20))

    # Etiqueta para mostrar el balance actual
    etiqueta_balance = tk.Label(
        frame_principal,
        text=f'Balance actual: ${balance_actual:.2f}',
        font=('Arial', 16),
        bg='#f0f0f0'
    )
    etiqueta_balance.pack(pady=(0, 20))

    # Frame para los botones de ingresos y egresos
    frame_botones = tk.Frame(frame_principal, bg='#f0f0f0')
    frame_botones.pack(pady=(0, 20))

    # Botón para ingresar ingresos
    boton_ingresar_ingreso = ttk.Button(
        frame_botones,
        text='Ingresar Ingreso',
        command=func_ingresar_ingreso,
        style="TButton",
    )
    boton_ingresar_ingreso.grid(row=0, column=0, padx=10, pady=10)

    # Botón para ingresar egresos
    boton_ingresar_egreso = ttk.Button(
        frame_botones,
        text='Ingresar Egreso',
        command=func_ingresar_egreso,
        style="TButton",
    )
    boton_ingresar_egreso.grid(row=0, column=1, padx=10, pady=10)

    # Botón de predicciones
    boton_recomendaciones = ttk.Button(
        frame_principal,
        text='Recomendaciones Personalizadas',
        command=lambda: mostrar_recomendaciones(usuario),
        style="TButton",
    )
    boton_recomendaciones.pack(pady=(10, 20))

    # Botón para predecir gastos futuros
    boton_prediccion_gastos = ttk.Button(
        frame_principal,
        text='Predecir Gastos Futuros',
        command=lambda: func_prediccion_gastos(usuario_actual),  # Asegúrate de pasar el usuario actual
        style="TButton",
    )
    boton_prediccion_gastos.pack(pady=10)

    # Botón para detectar anomalías
    boton_detectar_anomalias = ttk.Button(
        frame_principal,
        text='Detectar Anomalías en Gastos',
        command=lambda: func_detectar_anomalias(usuario),
        style="TButton",
    )
    boton_detectar_anomalias.pack(pady=10)

    # Etiqueta para seleccionar gráficas
    etiqueta_seleccion_grafica = tk.Label(
        frame_principal,
        text='Selecciona una gráfica \npara medir tu progreso financiero:',
        font=('Arial', 14),
        bg='#f0f0f0',  # Fondo claro
        fg='#333333'   # Color de texto oscuro
    )
    etiqueta_seleccion_grafica.pack(pady=(20, 10))

    # Opciones de gráficas
    opciones_graficas = ["Selecciona una gráfica",
        "Distribución de Gastos",
        "Proporción Ingresos vs. Egresos",
        "Evolución del Balance",
        "Comparación Mensual",
        "Relación Ingresos vs. Egresos",
        "Predicción de Balance",
        "Predicción de Ingresos",
        "Predicción de Egresos"
    ]

    # Variable para almacenar la selección del usuario
    seleccion_grafica = tk.StringVar(windowsmain)
    seleccion_grafica.set(opciones_graficas[0])  # Valor por defecto

    # Menú desplegable
    menu_graficas = tk.OptionMenu(
        frame_principal,
        seleccion_grafica,
        *opciones_graficas
    )
    menu_graficas.config(
        font=('Arial', 12),
        bg='#f0f0f0',  # Color verde
        fg='black',    # Texto blanco
        width=20
    )
    menu_graficas.pack(pady=10)

    # Botón para visualizar la gráfica seleccionada
    boton_ver_grafica_seleccionada = ttk.Button(
        frame_principal,
        text='Ver Gráfica Seleccionada',
        command=lambda: visualizar_grafica_seleccionada(seleccion_grafica.get()),
        style="TButton",
    )
    boton_ver_grafica_seleccionada.pack(pady=10)

    # Botón para cerrar sesión
    boton_cerrar_sesion = ttk.Button(
        frame_principal,
        text='Cerrar sesión',
        command=func_menuPrincipal,
        style="TButton",
    )
    boton_cerrar_sesion.pack(pady=20)

    # Guardar la referencia a la etiqueta de balance para poder actualizarla
    etiqueta_balance_actual = etiqueta_balance

def mostrar_recomendaciones(usuario):
    try:
        recomendaciones = obtener_recomendaciones_usuario(usuario)
        print(f"Recomendaciones generadas: {recomendaciones}")  # Depuración

        # Crear una nueva ventana para las recomendaciones
        ventana_recomendaciones = tk.Toplevel(windowsmain)
        ventana_recomendaciones.title("Recomendaciones Personalizadas")
        ventana_recomendaciones.geometry("600x400")
        ventana_recomendaciones.configure(bg='#f0f0f0')  # Fondo claro

        # Marco para organizar los elementos
        marco = tk.Frame(ventana_recomendaciones, padx=20, pady=20, bg='#f0f0f0')
        marco.pack(expand=True, fill='both')

        # Título de la ventana
        titulo = tk.Label(
            marco,
            text="💡 Recomendaciones para Optimizar tus Finanzas",
            font=('Arial', 14, 'bold'),
            fg='#2c3e50',  # Color de texto oscuro
            bg='#f0f0f0'   # Fondo claro
        )
        titulo.pack(pady=10)

        # Texto de las recomendaciones
        texto_recomendaciones = tk.Label(
            marco,
            text=recomendaciones,
            font=('Arial', 12),
            justify=tk.LEFT,
            wraplength=500,
            bg='#f0f0f0',  # Fondo claro
            fg='#333333'   # Color de texto oscuro
        )
        texto_recomendaciones.pack(pady=10, padx=10)

        # Botón para cerrar la ventana
        boton_cerrar = tk.Button(
            marco,
            text="Cerrar",
            command=ventana_recomendaciones.destroy,
            bg='#e74c3c',  # Color rojo
            fg='white',    # Texto blanco
            font=('Arial', 12, 'bold'),
            width=15
        )
        boton_cerrar.pack(pady=10)

    except Exception as e:
        messagebox.showerror("Error", f"No se pudieron cargar las recomendaciones: {str(e)}")


def func_registrarse():
    global usuarios

    def cargar_datos():
        global usuarios

        # Validar nombre y apellido
        nombre = entradas[0].get().strip()
        if not nombre or not all(caracter.isalpha() or caracter.isspace() for caracter in nombre):
            messagebox.showerror('Error', 'El nombre y apellido deben contener solo letras y espacios.')
            return

        # Validar cédula de identidad
        cedula = entradas[1].get().strip()
        if not cedula.isdigit() or len(cedula) < 6:
            messagebox.showerror('Error', 'La cédula de identidad debe ser un número válido (mínimo 6 dígitos).')
            return

        # Validar ocupación
        ocupacion = entradas[2].get().strip()
        if not ocupacion:
            messagebox.showerror('Error', 'La ocupación no puede estar vacía.')
            return

        # Validar ingreso mensual
        ingreso_mensual = entradas[3].get().strip()
        try:
            ingreso_mensual = float(ingreso_mensual)
            if ingreso_mensual <= 0:
                raise ValueError
        except ValueError:
            messagebox.showerror('Error', 'El ingreso mensual debe ser un número positivo.')
            return

        # Validar meta de ahorro
        meta_ahorro = entradas[4].get().strip()
        try:
            meta_ahorro = float(meta_ahorro)
            if meta_ahorro < 0:
                raise ValueError
        except ValueError:
            messagebox.showerror('Error', 'La meta de ahorro debe ser un número positivo.')
            return

        # Validar nombre de usuario
        nombre_usuario = entradas[5].get().strip()
        if not nombre_usuario or nombre_usuario in usuarios['Nombre de usuario'].values:
            messagebox.showerror('Error', 'El nombre de usuario no puede estar vacío o ya está en uso.')
            return

        # Validar correo electrónico
        correo = entradas[6].get().strip()
        if not re.match(r'^[\w\.-]+@[\w\.-]+\.\w+$', correo):
            messagebox.showerror('Error', 'El correo electrónico no es válido.')
            return

        # Validar contraseña
        contraseña = entradas[7].get().strip()
        if len(contraseña) < 6:
            messagebox.showerror('Error', 'La contraseña debe tener al menos 6 caracteres.')
            return
        
        # Hashear la contraseña antes de guardarla
        hashed_password = hash_password(contraseña)

        # Si todas las validaciones pasan, crear el usuario
        nuevo_usuario = {
            'Nombre del cliente': nombre,
            'I.D.': int(cedula),
            'Ocupación': ocupacion,
            'Ingreso Mensual': ingreso_mensual,
            'Meta de Ahorro': meta_ahorro,
            'Nombre de usuario': nombre_usuario,
            'Correo': correo,
            'Contraseña': hashed_password.decode('utf-8'),  # Guardar el hash como string
            'Balance': 0.00
        }

        # Guardar el usuario en el DataFrame y CSV
        nuevo_usuario_df = pd.DataFrame([nuevo_usuario])
        usuarios = pd.concat([usuarios, nuevo_usuario_df], ignore_index=True)
        usuarios.to_csv("usuarios.csv", index=False)

        messagebox.showinfo('Registro exitoso', '¡Usuario registrado con éxito!')
        func_menuPrincipal()

    # Limpiar la ventana antes de mostrar el formulario de registro
    for widget in windowsmain.winfo_children():
        widget.destroy()

    # Configuración de la ventana de registro
    windowsmain.configure(bg='#f0f0f0')
    etiqueta_registrarse_bienvenida = tk.Label(
        windowsmain, 
        text='Registrarse en CachiChén', 
        font=(montserrat_font.actual()["family"], 24, "bold"), 
        bg='#f0f0f0', 
        fg='#333333'
    )
    etiqueta_registrarse_bienvenida.pack(pady=5)

    # Descripción "Ingresa tu usuario y contraseña para iniciar sesión"
    descripcionr = tk.Label(
        windowsmain, 
        text="Comleta los campos señalados para registrarte en CachiChén", 
        font=(montserrat_font.actual()["family"], 12), 
        bg='#f0f0f0', 
        fg='#666666'
    )
    descripcionr.pack(pady=20)

    # Lista de etiquetas y campos de entrada
    etiquetas = [
        'Nombre y apellido: ',
        'Cédula de identidad: ',
        'Ocupación laboral: ',
        'Ingreso mensual: ',
        'Meta de ahorro: ',
        'Nombre de usuario: ',
        'Correo electrónico: ',
        'Contraseña: '
    ]
    
    # Frame para los campos de entrada
    frame_entradasr = tk.Frame(windowsmain, bg='#f0f0f0')
    frame_entradasr.pack(pady=10)

    entradas = []
    for etiqueta in etiquetas:
        frame = tk.Frame(frame_entradasr, bg='#f0f0f0')
        frame.pack(pady=5, anchor="w")
        label = tk.Label(
            frame, 
            text=etiqueta, 
            font=(montserrat_font.actual()["family"], 12), 
            bg='#f0f0f0', 
            fg='#333333'
        )
        label.pack(side="left", padx=10)
        framedos = tk.Frame(frame_entradasr, bg='#f0f0f0')
        framedos.pack(pady=5, anchor="e")
        entry = ttk.Entry(
            framedos, 
            font=(montserrat_font.actual()["family"], 12), 
            width=30
        )
        entry.pack(side="left", anchor="e")
        entradas.append(entry)

    frame_entradas_botonr = tk.Frame(windowsmain, bg='#f0f0f0')
    frame_entradas_botonr.pack(pady=10)

    # Botón "Crear usuario"
    boton_registrarse_enviarDatos = ttk.Button(
        frame_entradas_botonr, 
        text='Crear usuario', 
        command=cargar_datos, 
        style="TButton"
    )
    boton_registrarse_enviarDatos.pack(side="left", padx=10)

    # Botón "Volver a atrás"
    boton_registrarse_atras = ttk.Button(
        frame_entradas_botonr, 
        text='Volver a atrás', 
        command=func_menuPrincipal, 
        style="TButton"
    )
    boton_registrarse_atras.pack(side="left")

    # Configurar el estilo de los botones
    style = ttk.Style()
    style.configure("TButton", font=(montserrat_font.actual()["family"], 12))

def func_salir():
    windowsmain.destroy()

# Crear la ventana principal
windowsmain = tk.Tk()
windowsmain.geometry("500x900")  # Tamaño de la ventana
windowsmain.title("CachiChén")   # Título de la ventana
windowsmain.configure(bg='#f0f0f0')  # Fondo gris claro
windowsmain.resizable(1,1)   # Título de la ventana
# Definir la fuente Montserrat
try:
    montserrat_font = font.Font(family="Montserrat", size=12)  # Usar Montserrat si está instalada
except:
    montserrat_font = font.Font(family="Helvetica", size=12)  # Usar Helvetica como alternativa

# Crear un Canvas como contenedor principal
canvas = tk.Canvas(windowsmain)
canvas.pack(side="left", fill="both", expand=True)

# Añadir una Scrollbar vertical
scrollbar = ttk.Scrollbar(windowsmain, orient="vertical", command=canvas.yview)
scrollbar.pack(side="right", fill="y")

# Configurar el Canvas para que use la Scrollbar
canvas.configure(yscrollcommand=scrollbar.set)
canvas.bind(
    "<Configure>",
    lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
)

# Crear un Frame dentro del Canvas para contener los widgets
scrollable_frame = ttk.Frame(canvas)
canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")

# Añadir contenido al Frame (esto es solo un ejemplo)
for i in range(50):
    label = ttk.Label(scrollable_frame, text=f"Elemento {i}")
    label.pack(pady=5)


# Función del menú principal
def func_menuPrincipal():
    # Limpiar la ventana antes de mostrar el menú principal
    for widget in windowsmain.winfo_children():
        widget.destroy()

    # Crear un estilo para los widgets ttk
    
    estilo = ttk.Style()
    estilo.configure("Titulo.TLabel", font=('Arial', 24), background='#f0f0f0', foreground='#333333')
    estilo.configure("Subtitulo.TLabel", font=('Arial', 18), background='#f0f0f0', foreground='#333333')
    estilo.configure("Texto.TLabel", font=('Arial', 12), background='#f0f0f0', foreground='#666666')
    estilo.configure("TButton", font=('Arial', 12), padding=5)

    # Título "Bienvenido a CachiChén"
    titulo = ttk.Label(
        windowsmain, 
        text="Bienvenido a CachiChén", 
        style="Titulo.TLabel"
    )
    titulo.pack(pady=20)

    # Subtítulo "Inicia Sesión"
    subtitulo = ttk.Label(
        windowsmain, 
        text="Inicia Sesión", 
        style="Subtitulo.TLabel"
    )
    subtitulo.pack(pady=10)

    # Descripción "Ingresa tu usuario y contraseña para iniciar sesión"
    descripcion = ttk.Label(
        windowsmain, 
        text="Ingresa tu usuario y contraseña para iniciar sesión", 
        style="Texto.TLabel"
    )
    descripcion.pack(pady=10)

    # Frame para los campos de entrada
    frame_entradas = ttk.Frame(windowsmain)
    frame_entradas.pack(pady=10)

    # Campo de entrada para el usuario
    label_usuario = ttk.Label(
        frame_entradas, 
        text="Usuario", 
        style="Texto.TLabel"
    )
    label_usuario.grid(row=0, column=0, padx=10, pady=5, sticky="w")

    usuario = ttk.Entry(
        frame_entradas, 
        font=('Arial', 12), 
        width=30
    )
    usuario.grid(row=0, column=1, padx=10, pady=5)

    # Campo de entrada para la contraseña
    label_contraseña = ttk.Label(
        frame_entradas, 
        text="Contraseña", 
        style="Texto.TLabel"
    )
    label_contraseña.grid(row=1, column=0, padx=10, pady=5, sticky="w")

    contraseña = ttk.Entry(
        frame_entradas, 
        font=('Arial', 12), 
        width=30, 
        show="*"  # Ocultar la contraseña
    )
    contraseña.grid(row=1, column=1, padx=10, pady=5)

    # Botón "Ingresar"
    boton_ingresar = ttk.Button(
        windowsmain, 
        text="Ingresar", 
        style="TButton",
        command=lambda: func_autenticacion(usuario.get(), contraseña.get())
    )
    boton_ingresar.pack(pady=20)

    # Texto "¿Aún no tienes cuenta? | Registrarse"
    frame_registro = ttk.Frame(windowsmain)
    frame_registro.pack(pady=10)

    texto_registro = ttk.Label(
        frame_registro, 
        text="¿Aún no tienes cuenta?", 
        style="Texto.TLabel"
    )
    texto_registro.pack(side="left")

    # Botón "Registrarse" (enlace)
    boton_registrarse = ttk.Button(
        frame_registro, 
        text="Registrarse", 
        style="TButton",
        command=func_registrarse
    )
    boton_registrarse.pack(side="left", padx=5)

# Función de autenticación
def func_autenticacion(usuario, contraseña):
    global usuario_actual, login_attempts

    # Verificar intentos fallidos
    if login_attempts >= MAX_LOGIN_ATTEMPTS:
        messagebox.showerror('Error', 'Demasiados intentos fallidos. Intente más tarde.')
        return

    # Buscar el usuario en el DataFrame
    if usuario in usuarios['Nombre de usuario'].values:
        # Obtener el hash de la contraseña del usuario
        hashed_password = usuarios.loc[usuarios['Nombre de usuario'] == usuario, 'Contraseña'].values[0]

        # Verificar la contraseña
        if verify_password(contraseña, hashed_password.encode('utf-8')):  # Convertir a bytes
            login_attempts = 0  # Reiniciar contador
            func_menuSesionIniciada(usuario)  # Llamar a la función de sesión iniciada
        else:
            login_attempts += 1
            messagebox.showerror('Error', f"Contraseña incorrecta. Intentos restantes: {MAX_LOGIN_ATTEMPTS - login_attempts}")
    else:
        messagebox.showerror('Error', "Usuario no encontrado")



# Llamar a la función para mostrar el menú principal
func_menuPrincipal()

# Ejecutar la aplicación
windowsmain.mainloop()


Ejecutando func_menuSesionIniciada para el usuario: lorean04
Ejecutando func_menuSesionIniciada para el usuario: lorean04
